<a href="https://colab.research.google.com/github/DataJenius/NLPEncodingExperiment/blob/main/python/NLPEncodingExperiment_get_BERT_pt_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################
# encode all of our comments as 768-dimension BERT embeddings
# https://stackoverflow.com/questions/65627663/bert-extracting-cls-embedding-from-multiple-outputs-vs-single?rq=1
# https://stackoverflow.com/questions/59330597/bert-sentence-embeddings-how-to-obtain-sentence-embeddings-vector


In [ ]:
###########################################
# install all dependencies
!pip install transformers

In [ ]:
###########################################
# load dependencies
from transformers import AutoTokenizer, TFBertModel

#, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import numpy as np
import os
from google.colab import files

In [ ]:
###############################################
# use BERT tokenizer & classifier
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', trainable=False, num_labels=2)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
#################################################################################
# our labelled, raw comment data is on github
all_files = ['https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group1.csv',
             'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group2.csv',
             'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group3.csv',
             'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group4.csv',
             'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group5.csv']

# concat into a single dataframe and shuffle the contents
df_from_each_file = (pd.read_csv(f) for f in all_files)
df_all_data = pd.concat(df_from_each_file, ignore_index=True)
df_all_data = df_all_data.sample(frac=1).reset_index(drop=True)
print(df_all_data.shape)
print(df_all_data.head())

(10000, 8)
   msg_id  token_count  my_group   my_role  label       source  \
0    9923           22         5      test      0  /r/StarWars   
1   38879           58         4  validate      1      /r/lotr   
2   29415           29         4  validate      1      /r/lotr   
3    4039           18         2     train      0  /r/StarWars   
4    1432           17         2     train      0  /r/StarWars   

                                            raw_text  \
0  Yep, RR directing was very bad, but there were...   
1  Way to ignore all context and arguments in goo...   
2  Again, you're basing this entire opinion on a ...   
3  And the spice syndicate, don't they have like ...   
4  I think a limited run of 5-8 episodes could be...   

                                          clean_text  
0  yep rr directing was very bad but there were a...  
1  way to ignore all context and arguments in goo...  
2  again youre basing this entire opinion on a te...  
3  and the spice syndicate dont the

In [ ]:
#################################################################################
# tokenize all 10,000 comments
tokenizer_output = bert_tokenizer(df_all_data["raw_text"].tolist(), 
                                  truncation=True,                                  
                                  max_length=500, 
                                  padding='max_length',
                                  return_tensors='np')

# put tokenized comments into np array
tokenizer_output_np = np.array([
    tokenizer_output['input_ids'], 
    tokenizer_output['attention_mask']
])
print(tokenizer_output_np.shape)

(2, 10000, 500)


In [ ]:
#################################################################################
# forgive this goofy loop, I wanted to be able to sanity-check at each step
# loop through all 10,000 comments
# takes amount 2 hours to get BERT embeddings from them all
df_embeddings = pd.DataFrame()
j = 0
for i in range(0,10000,1):
  j += 1
  print(i, j)

  # get [CLS] embedding for this comment
  my_input = [tokenizer_output_np[0][i:i+1], tokenizer_output_np[1][i:i+1]] # Shape : (1,500) (1,500)
  my_output = bert_model.predict(my_input)
  my_cls = my_output[0][:,0,:] # Shape : (1, 768)

  # put it into a dataframe for use elsewhere
  values = my_cls.squeeze().tolist()
  df_my_embeddings = pd.DataFrame({'value': values}).transpose()
  df_my_embeddings["msg_id"] = df_all_data.iloc[i]["msg_id"]
  df_my_embeddings.index=[i]
  df_embeddings = df_embeddings.append(df_my_embeddings)   

  # save every 1000 records
  if j == 1000:

      # save results to local CSV
      file_name = 'BERT_embeddings_i'+str(i)+'.csv'
      df_embeddings.to_csv(file_name, index=False) 
      files.download(file_name)
      print("\n\nsave progress...\n\n",df_embeddings.head(10))      

      # reset df and j counter
      df_embeddings = pd.DataFrame()
      j=0

  #if i == 267:
  #  break

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 60
60 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
99 100
100 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 111
111 112
112 113
113 114
114 115
115 116
116 117
117 118
118 119
119 120
120 121
121 122
122 123
123 124
124 125
125 126
126 127
127 128
128 129
129 130
130 131
131 132
132 133
133 134
134 135
135 136
136 137
137 138
138 139
139 140
140 141
141 142
142 143
143 144
144 145
145 146
146 147
147 148
148 149
149 150
150 151
151 152
15

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



save progress...

           0         1         2         3         4         5         6  \
0  0.007723 -0.025574  0.175493 -0.038083 -0.329894 -0.110158  0.157542   
1  0.341545  0.185957 -0.088983 -0.016812 -0.402677 -0.264315  0.137367   
2  0.051385 -0.205393 -0.067455  0.084979 -0.575922 -0.201654  0.131130   
3  0.244150 -0.063882  0.190421  0.056093 -0.722250 -0.145684  0.059774   
4  0.007838 -0.398030  0.048651 -0.297497 -0.310664 -0.298068  0.001150   
5 -0.205928 -0.681554 -0.079084  0.202485 -0.698614 -0.418531 -0.434251   
6  0.436328 -0.608490  0.402088  0.153449 -0.612226 -0.604861  0.129110   
7 -0.389868 -0.494005 -0.253824  0.305682 -0.621787 -0.361605 -0.021366   
8  0.184861 -0.369838 -0.019996  0.031214 -0.322090 -0.491898 -0.028071   
9  0.248473 -0.073738  0.218109 -0.225075 -0.579103 -0.375420  0.429654   

          7         8         9  ...       759       760       761       762  \
0  0.624229  0.160256 -0.574947  ... -0.594470  0.141395 -0.278089  0.137

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>



save progress...

              0         1         2         3         4         5         6  \
8000  0.210978 -0.365924  0.241585 -0.013909 -0.389829 -0.073530  0.229285   
8001  0.275279  0.134905  0.284627 -0.192636 -0.186372 -0.379877  0.330736   
8002  0.639710  0.224408 -0.293656  0.104530 -0.289828 -0.494360  0.575686   
8003  0.004174 -0.142704  0.006451 -0.317812 -0.272723 -0.395661  0.105163   
8004  0.600863 -0.205149  0.044715  0.101431 -0.497202 -0.501527  0.100017   
8005  0.183072 -0.352979 -0.378455 -0.317445 -0.426657 -0.219283  0.224555   
8006  0.018982  0.113867  0.047510 -0.163166 -0.363451 -0.137090  0.363874   
8007  0.288142  0.022036 -0.203982  0.200026 -0.326876 -0.709024  0.112925   
8008  0.491562  0.314121 -0.227868 -0.163597 -0.224419 -0.350725  0.176580   
8009 -0.505444 -0.419083 -0.140344  0.452833 -0.418154 -0.410523  0.018005   

             7         8         9  ...       759       760       761  \
8000  0.518172  0.063919 -0.084734  ... -0.2240

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



save progress...

              0         1         2         3         4         5         6  \
9000 -0.048557 -0.168804 -0.014584 -0.418477 -0.284797 -0.151885  0.489796   
9001  0.165299  0.011111  0.028682 -0.177584 -0.507271 -0.711152  0.365144   
9002  0.252007 -0.272850  0.332500 -0.119530  0.038965 -0.337528 -0.030102   
9003 -0.707969 -0.373064 -0.894306  0.394536 -0.836453 -0.096555  0.037092   
9004  0.033425 -0.310231 -0.155892  0.161684 -1.023448 -0.266278  0.013619   
9005 -0.349715 -0.277529 -0.001990  0.324522 -0.658866 -0.010215  0.277428   
9006 -0.155868 -0.029656  0.444205  0.145694 -0.370254 -0.117515  0.282467   
9007  0.342806  0.172664 -0.310461 -0.343477 -0.573348 -0.418449  0.365239   
9008  0.290053 -0.205997 -0.313159 -0.044952 -0.130626 -0.357653  0.333982   
9009  0.056056 -0.120343  0.520524 -0.373062 -0.084793 -0.396445  0.140631   

             7         8         9  ...       759       760       761  \
9000  0.546657  0.134113 -0.123901  ... -0.3701